In [73]:
from flask import Flask
from flask import jsonify
import os
import sys
import scipy
import numpy as np
import pandas as pd
from sklearn import metrics, preprocessing
from modules import ml_create_target_df, get_top_probable_tags, mean_local_score, f1_local_score_accuracy, text_processing, text_processing_advanced, local_score
from sklearn.externals import joblib

In [74]:
def prediction_simple():
    
    current_path = os.getcwd()
    parent_path = os.path.dirname(current_path)
    # I. On récupère les données utilisateur
    title_input = request.form['Title']
    body_input = request.form['Body']

    body_title_input = title_input + " "+ body_input

# II. Nettoyage	du texte.
    body_title_input_cleaned = text_processing_advanced(body_title_input)
    
# III. Chargement des variables utiles liées à notre jeu d'entrainement
    vectorizer_train = joblib.load(current_path + "/mysite/pipeline/vectorizer.pkl")
    count_tags_df = joblib.load(current_path + "/mysite/pipeline/count_tags_df.pkl")
    most_popular_tag = joblib.load(current_path + "/mysite/pipeline/most_popular_tag.pkl")
    unique_tags_serie = joblib.load(current_path + "/mysite/pipeline/unique_tags_serie.pkl")
    unique_tags = joblib.load(current_path + "/mysite/pipeline/unique_tags.pkl")
    train_selected_tags = joblib.load(current_path + "/mysite/pipeline/selected_tags.pkl")
    list_top_tags = joblib.load(current_path + "/mysite/pipeline/list_top_tags.pkl")

    # IV. Chargement des classifieurs (logistical regression)
    training_classifiers = []
    for tag in train_selected_tags:
        training_classifiers.append(joblib.load(parent_path + "/classifiers/logistical_regression/logreg_" + tag +".pkl"))
    
    body_title_input_cleaned = body_title_input_cleaned.split("0")

    # V. Application du vectorizer_train de notre training
    X_input = vectorizer_train.transform(body_title_input_cleaned)

    # VI. Prédiction de la probabilité
    print("Prédiction de la probabilité pour chaque question d'appartenir à Tag donné en cours de traitement...")
    input_predictions = []
    count = 1
    zero_array = np.zeros((X_input.shape[0]))
    for clf in training_classifiers:
        prediction = clf.predict_proba(X_input)

        if prediction.shape[1] == 2:
            input_predictions.append(prediction[:,1])
        elif prediction.shape[1] == 1:
            input_predictions.append(zero_array)
        count += 1

    X_input_predictions_df = pd.DataFrame(np.array(input_predictions).T, columns = train_selected_tags)
    TAGS = get_top_probable_tags(X_input_predictions_df.iloc[0], train_selected_tags)

In [84]:
TAGS

'networking/sockets/c#/java/c++'